# Supply Chain Data Preprocessing & Analysis

# 1. Initial Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy import stats

# Load dataset
file_path = "../Data/dynamic_supply_chain_logistics_dataset.csv"
df = pd.read_csv(file_path)

# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Display initial information
print("Dataset Shape:", df.shape)
print("\nColumn Names:")
print(df.columns.tolist())
print("\nData Types:")
df.info()

Dataset Shape: (32065, 26)

Column Names:
['timestamp', 'vehicle_gps_latitude', 'vehicle_gps_longitude', 'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability', 'order_fulfillment_status', 'weather_condition_severity', 'port_congestion_level', 'shipping_costs', 'supplier_reliability_score', 'lead_time_days', 'historical_demand', 'iot_temperature', 'cargo_condition_status', 'route_risk_level', 'customs_clearance_time', 'driver_behavior_score', 'fatigue_monitoring_score', 'disruption_likelihood_score', 'delay_probability', 'risk_classification', 'delivery_time_deviation']

Data Types:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32065 entries, 0 to 32064
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   timestamp                        32065 non-null  object 
 1   

### Initial Data Analysis Results

The dataset contains supply chain logistics data with the following characteristics:
- **Size**: 32,065 records with 26 features
- **Features**: Mix of numerical and categorical variables
- **Time Range**: Spans multiple time periods
- **Key Metrics**: Includes various logistics and performance indicators

**Key Features**:
- Temporal: timestamp
- Location: vehicle_gps_latitude, vehicle_gps_longitude
- Performance: delivery_time_deviation, eta_variation_hours
- Risk: disruption_likelihood_score, route_risk_level
- Operations: warehouse_inventory_level, loading_unloading_time

# 2. Data Cleaning and Type Conversion

In [ ]:
# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Check for missing values and duplicates
missing_values = df.isnull().sum()
missing_percentages = (missing_values / len(df)) * 100
duplicates = df.duplicated().sum()

print('Missing Values Analysis:')
print(missing_percentages[missing_percentages > 0])
print(f'\nDuplicate rows: {duplicates}')

# Display basic statistics
print('\nNumerical Columns Statistics:')
print(df.select_dtypes(include=['float64', 'int64']).describe())

Missing Values Analysis:
Series([], dtype: float64)

Duplicate rows: 0

Numerical Columns Statistics:
       vehicle_gps_latitude  vehicle_gps_longitude  fuel_consumption_rate  \
count          32065.000000           32065.000000           32065.000000   
mean              38.023589             -90.116648               8.011735   
std                6.917909              17.369244               4.264960   
min               30.000000            -119.999998               5.000000   
25%               31.280550            -106.253913               5.019984   
50%               36.413820             -86.293414               5.636036   
75%               44.453655             -73.079367               9.669944   
max               50.000000             -70.000000              19.999875   

       eta_variation_hours  traffic_congestion_level  \
count         32065.000000              3.206500e+04   
mean              2.893068              4.991493e+00   
std               2.274044          

### Data Cleaning Results
- Timestamp conversion completed successfully
- No missing values identified in any columns
- No duplicate records found
- Numerical columns show reasonable value distributions

# 3. Feature Engineering

In [ ]:
# Create derived features
df['date'] = df['timestamp'].dt.date
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.day_name()
df['month'] = df['timestamp'].dt.month

# Calculate total risk score (combining multiple risk factors)
df['total_risk_score'] = (df['disruption_likelihood_score'] + 
                        df['route_risk_level'] + 
                        df['delay_probability']) / 3

# Calculate delivery efficiency score
df['delivery_efficiency'] = 100 - (abs(df['delivery_time_deviation']) * 10 + 
                                 abs(df['eta_variation_hours']) * 5)

# Categorize delays
df['delay_category'] = pd.cut(df['delivery_time_deviation'],
                            bins=[-float('inf'), -2, -0.5, 0.5, 2, float('inf')],
                            labels=['Very Early', 'Early', 'On Time', 'Late', 'Very Late'])

# Clean and normalize scores to 0-100 scale where needed
cols_to_normalize = ['driver_behavior_score', 'fatigue_monitoring_score', 
                    'supplier_reliability_score']

for col in cols_to_normalize:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min()) * 100

# Calculate distance between consecutive points with improved handling
df['distance'] = np.sqrt(
    (df['vehicle_gps_latitude'].diff())**2 + 
    (df['vehicle_gps_longitude'].diff())**2
)

# Fill first row's distance with the mean of next 5 rows and round all distances
first_rows_mean = df['distance'][1:6].mean()
df['distance'] = df['distance'].fillna(first_rows_mean).round(3)

print("Data preprocessing completed. New features created:")
print("\nShape of dataset:", df.shape)
print("\nNew columns added:", 
      [col for col in df.columns if col not in ['timestamp', 'vehicle_gps_latitude', 'vehicle_gps_longitude', 
       'fuel_consumption_rate', 'eta_variation_hours', 'traffic_congestion_level', 
       'warehouse_inventory_level', 'loading_unloading_time', 'handling_equipment_availability',
       'order_fulfillment_status', 'weather_condition_severity', 'port_congestion_level',
       'shipping_costs', 'supplier_reliability_score', 'lead_time_days', 'historical_demand',
       'iot_temperature', 'cargo_condition_status', 'route_risk_level', 'customs_clearance_time',
       'driver_behavior_score', 'fatigue_monitoring_score', 'disruption_likelihood_score',
       'delay_probability', 'risk_classification', 'delivery_time_deviation']])

Data preprocessing completed. New features created:

Shape of dataset: (32065, 34)

New columns added: ['date', 'hour', 'day_of_week', 'month', 'total_risk_score', 'delivery_efficiency', 'delay_category', 'distance']


### Feature Engineering Results
- Created temporal features: date, hour, day_of_week, month
- Added composite metrics: total_risk_score, delivery_efficiency
- Created delay categories for better analysis
- Normalized relevant scores to 0-100 scale
- Added distance calculations between consecutive points

# 4. Performance Metrics Creation

In [ ]:
# Add delivery performance score
df['delivery_performance'] = df.apply(lambda x: 
    100 * (1 - abs(x['delivery_time_deviation'])/10) * 
    (1 - x['delay_probability']) * 
    (1 - x['disruption_likelihood_score']), axis=1)

# Add cost efficiency metric
df['cost_efficiency'] = df['shipping_costs'].mean() / df['shipping_costs'] * 100

### Performance Metrics Results
- Created delivery_performance score combining multiple factors
- Added cost_efficiency metric based on shipping costs
- New metrics provide better insights into supply chain performance

# 5. Outlier Detection and Handling

In [ ]:
# Handle outliers using IQR method
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower_bound, upper_bound)

numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    handle_outliers(df, col)

# Handle decimals for numeric columns
# Add after the outlier handling section and before final validation

# Round numeric measurements and scores to appropriate decimal places
decimal_handling = {
    # GPS coordinates (higher precision needed)
    'vehicle_gps_latitude': 6,
    'vehicle_gps_longitude': 6,
    # Distance and time measurements
    'distance': 3,
    'eta_variation_hours': 2,
    'lead_time_days': 1,
    'loading_unloading_time': 2,
    'customs_clearance_time': 2,
    # Scores and metrics (2 decimals for percentage-based metrics)
    'driver_behavior_score': 2,
    'fatigue_monitoring_score': 2,
    'supplier_reliability_score': 2,
    'delivery_efficiency': 2,
    'total_risk_score': 2,
    'cost_efficiency': 2,
    'delivery_performance': 2,
    # Other measurements
    'fuel_consumption_rate': 2,
    'shipping_costs': 2,
    'iot_temperature': 1,
    # Probabilities and indices (3 decimals for more precise probability values)
    'delay_probability': 3,
    'disruption_likelihood_score': 3
}

# Apply rounding to specified columns
for col, decimals in decimal_handling.items():
    if col in df.columns:
        df[col] = df[col].round(decimals)

print("Decimal handling completed for numeric columns")

Decimal handling completed for numeric columns


### Outlier Handling Results
- Applied IQR method to handle outliers
- Outliers capped at 1.5 * IQR boundaries
- Maintained data integrity while removing extreme values
- Treated all numeric columns for consistency

# 6. Final Validation and Export

In [ ]:
# Add data validation checks
validation_results = {
    'timestamp_range': df['timestamp'].min() >= pd.Timestamp('2021-01-01'),
    'gps_coords_valid': all([-90 <= df['vehicle_gps_latitude'].max() <= 90,
                            -180 <= df['vehicle_gps_longitude'].max() <= 180]),
    'scores_normalized': all(df[cols_to_normalize].max() <= 100)
}

print('\nValidation Results:')
for check, result in validation_results.items():
    print(f'{check}: {"Passed" if result else "Failed"}')

# Save the cleaned dataset
cleaned_file_path = "../Data/cleaned_supply_chain_logistics_dataset.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned dataset saved to {cleaned_file_path}")


Validation Results:
timestamp_range: Passed
gps_coords_valid: Passed
scores_normalized: Passed
Cleaned dataset saved to ../Data/cleaned_supply_chain_logistics_dataset.csv
